In [39]:
from lenskit.algorithms import Recommender, als, funksvd, item_knn, user_knn
from lenskit import batch, topn, util, crossfold as xf
import matplotlib.pyplot as plt
from lenskit.metrics.predict import rmse
from matplotlib.dates import DateFormatter
import pandas as pd
import joblib

from mlcomp.data.load import load_regression_train

In [ ]:
df = load_regression_train()

print(df)
# remove everything before 01-01-2014
#df = df.loc[df['timestamp'] > 1388530800 * 1000]
# remove everything before 01-01-2018
#df = df.loc[df['timestamp'] > 1514764800000]
# remove everything before 01-01-2011
df = df.loc[df['timestamp'] > 1262300400 * 1000]


print(df)
#df = df[:10000]



In [ ]:


ttpair_gen = xf.partition_users(df, 2, xf.SampleFrac(0.15))
ttpair = ttpair_gen.__next__()

train = ttpair.train
test = ttpair.test

X_val = test.drop(columns=['rating'])
y_val = test['rating']

#joblib.dump(train, 'yee_train.joblib')
#joblib.dump(test, 'yee_test.joblib')

In [41]:
model = als.BiasedMF(100, iterations=200)
model.fit(df)


joblib.dump(model, "models/M8-bmf.joblib")

['models/M8-bmf.joblib']

In [ ]:
pred = batch.predict(model, X_val)


In [ ]:

# lenskit messed up the index but keeps the order, so we can fix it like this:
pred.index = X_val.index

print(pred)

rmse(pred['prediction'], y_val) 

### Model Doc:

#### models/M1-bmf.joblib:
- Drop everything before 01-01-2014
- BiasedMF(50)
- 85/15 TTS
- Val RMSE: 0.8918094342855158
- Kaggle: 1.17477

#### models/M2-fsvd.joblib:
- Drop everything before 01-01-2014
- FunkSVD(50)
- 85/15 TTS
- Val RMSE: 0.8955200550427233
- Kaggle: 1.19404

Further optimization was done inside WSL, see the WSL Repo for more.

#### models/M3-bmf.joblib:
- Drop everything before 01-01-2014
- BiasedMF using Config from Run 1 (see WSL)
- 85/15 TTS
- Val RMSE: 0.893102549930063
- Kaggle: 1.17263

#### models/M4-itemknn.joblib:
- error


#### models/M5-userknn.joblib:
- !!! 38952/39194 NaN !!!

- Drop everything before 01-01-2014
- ItemItem(nnbrs=50, feedback="explicit")
- 85/15 TTS
- Val RMSE: 1.147419143612323
- Kaggle: 

#### models/M6-bmf.joblib:
- Drop everything before 01-01-2014
- BiasedMF(200)
- 85/15 TTS
- Val RMSE: 0.8882260413979319
- Kaggle: 1.17052


#### models/M7-bmf.joblib:
- Drop everything before 01-01-2011
- BiasedMF(200)
- - 85/15 TTS
- Val RMSE: 0.8761605756905748
- Kaggle: 1.17112
- -all suffix: Train on all data instead of TTS; on kaggle: 1.17094

#### models/M8-bmf.joblib:
- Drop everything before 01-01-2011
- BiasedMF(100, iteration=100)
- Training on all the data
- Kaggle: 
